## Imports

In [1]:
from dataloader.load_dt_simulated import load_dt_simulated_dataset

# Simulated dataset generation

The complete synthetic dataset consists of diffusion weighted data (b-values, gradient orientations, diffusion weighted signals) and the SH expansion coefficients of the fODF which is the target. The simulation starts with sampling random fibre orientations and the corresponding volume fractions for predetermined number of fibre populations. Then given these directions and volume fractions fODF SH coefficients are generated as descibed before. Before the corresponding DW signals can be generated, the eigenvectors of the diffusion tensor need to be generated. Since the eigenvalues are provided as simulation parameters the first eigenvector is chosen to be the fibre population orientation. Then other two random vectors are sampled and by the process of Gram-Schmidt orthonormalization are transformed so that together with the first eigenvector they form an orthonormal basis. These vectors are then used as the eigenvectors of the diffusion tensor model and the corresponding diffusion-weighted signals are produced.

The following cell simulates dataset of 1000 voxels each with 2 fibre populations and maximum SH degree of 8.

In [2]:
dataset = load_dt_simulated_dataset(dataset_size=1000, number_of_fibre_populations=2, max_degree=8, vary_gradients=False,seed=1)

The following three sections show synthetic data for one voxel.

## SH coefficients of fODF

In [3]:
dataset[0][0]

array([ 0.28209479,  0.06192353, -0.2712074 , -0.2165312 , -0.01007259,
       -0.15952789, -0.43814622, -0.02025965,  0.03873198,  0.36915473,
        0.02807401, -0.00309869,  0.00380469,  0.27289658, -0.14681116,
        0.29809931,  0.14021601, -0.06531286,  0.05637124, -0.11197503,
       -0.30197654,  0.16974487,  0.03811946,  0.1214478 , -0.22737339,
        0.0199729 , -0.53834829,  0.05822074,  0.19546029, -0.21094007,
        0.13877934, -0.07425833,  0.34068434, -0.09310615,  0.12015208,
        0.10564028, -0.29106338, -0.07875803, -0.17599889,  0.05917967,
        0.06587996,  0.37341549,  0.12791213,  0.05426927, -0.25862614])

## b-vals, gradient orientations, signals

In [4]:
dataset[1][0]

(array([1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000,
        1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000,
        1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000,
        1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000,
        1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000,
        1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000,
        1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000,
        1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000,
        1000, 1000]),
 array([[ 0.36353657, -0.79057113, -0.61636162,  0.47329009, -0.82269029,
          0.89697745, -0.94988454, -0.16577181, -0.81933594, -0.8108305 ,
          0.87454161,  0.91075499, -0.29634718,  0.07737383, -0.0756303 ,
          0.18488546,  0.53113688, -0.67804166,  0.52939376, -0.0155802 ,
          0.0529998 , -0.9090384 ,  0.12865447,  0.19786134, -0.71456792,
          0.5073

## Fibre orientations

In [5]:
dataset[2][0]

array([[ 0.36353657,  0.86429949,  0.34760259],
       [-0.79057113,  0.54924163,  0.27079683]])

## Saving the datasets